<a href="https://colab.research.google.com/github/ShermanZaa555/CS385_project/blob/modeling/cs385_modeling_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
FILE_PATH = '/content/drive/MyDrive/dataset/Dummy1.csv'
df = pd.read_csv(FILE_PATH)
df.head()

,Order_Source,Order_ID,Order_Date,Customer_ID,Customer_Name,Address,ZIP,Tel,Product_ID,Product_Price,Quantity,Promotion,TotalPrice
0,Lazada,3635,23/04/2022 11:41,69302,Emmanuel Stephenson,"328-1195 Mi, Rd.",94619,037-026-6153,THT0008,495,4,Yes,2015
1,Lazada,9120,02/12/2022 02:12,59388,Salvador Mayer,Ap #697-1332 In Street,97427,039-654-8269,THT0005,358,3,Yes,1109
2,Lazada,2770,22/06/2022 18:36,89382,Cameron Greene,Ap #664-2817 Tincidunt Ave,64621,013-312-8668,THT0017,305,3,Yes,950
3,Facebook,4199,06/09/2022 05:11,30769,Hayes Keith,"P.O. Box 347, 823 Suspendisse St.",52368,017-473-8603,THT0011,476,3,No,1463
4,Shopee,3248,01/02/2023 20:34,27330,Lisandra Page,415-469 Interdum. Rd.,72859,075-888-5181,THT0003,254,2,Yes,543


In [ ]:
df_clean = df.copy()

In [ ]:
df_clean = df_clean.drop('TotalPrice', axis=1)

In [ ]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df_clean)

Order_Source :  5
Order_ID :  489
Order_Date :  500
Customer_ID :  499
Customer_Name :  500
Address :  500
ZIP :  500
Tel :  500
Product_ID :  20
Product_Price :  219
Quantity :  5
Promotion :  2


In [ ]:
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['Product_Price']

In [ ]:
df_clean['Order_Date'].min(), df_clean['Order_Date'].max()

('01/01/2022 19:15', '31/12/2022 14:16')

In [ ]:
import datetime as dt
NOW = dt.datetime(2023,1,1)
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'])

In [ ]:
df_clean = df_clean.sort_values('Order_Date').reset_index()

In [ ]:
df_clean = df_clean.drop('index', axis=1)

In [ ]:
rfmTable = df_clean.groupby('Customer_ID').agg({'Order_Date': lambda x: (NOW - x.max()).days, 'Order_ID': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

In [ ]:
rfmTable['Order_Date'] = rfmTable['Order_Date'].astype(int)
rfmTable.rename(columns={'Order_Date': 'recency', 
                         'Order_ID': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [ ]:
rfmTable.head()

,recency,frequency,monetary_value
Customer_ID,,,
10627,296,1,996
11055,490,1,1460
11205,217,1,1012
11554,98,1,656
12137,128,1,1200


In [ ]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [ ]:
segmented_rfm = rfmTable.copy()

In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
Customer_ID,,,,,,
10627,296,1,996,3,4,3
11055,490,1,1460,4,4,2
11205,217,1,1012,2,4,3
11554,98,1,656,2,4,4
12137,128,1,1200,2,4,2


In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
Customer_ID,,,,,,,
10627,296,1,996,3,4,3,343
11055,490,1,1460,4,4,2,442
11205,217,1,1012,2,4,3,243
11554,98,1,656,2,4,4,244
12137,128,1,1200,2,4,2,242


In [ ]:
segmented_rfm['RFMScore'].value_counts()

442    41
244    36
144    36
342    35
243    35
143    35
341    33
441    32
343    31
141    30
241    28
444    28
242    26
344    25
443    24
142    23
111     1
Name: RFMScore, dtype: int64

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']=='441'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
Customer_ID,,,,,,,
19395,558,1,2470,4,4,1,441
74728,722,1,2400,4,4,1,441
51237,570,1,2345,4,4,1,441
35996,564,1,2315,4,4,1,441
76134,467,1,2255,4,4,1,441
63300,533,1,2250,4,4,1,441
94837,527,1,2195,4,4,1,441
23812,632,1,2110,4,4,1,441
51413,451,1,2105,4,4,1,441
